# Chocolate Database - Data Munging Notebook
### Sources: USDA Food Composition Website, CSV files, Twitter

In [61]:
# Importing relevant libraries
import os
from bs4 import BeautifulSoup
import urllib3
import pandas as pd
import re
from itertools import repeat
import csv
import numpy as np
import time
from tweepy import API
from tweepy import Cursor
from tweepy import TweepError
from tweepy import OAuthHandler


# importing Twitter API keys from file
import t_credentials as twitter_credentials

## Part I - Scraping USDA Food Composition Database Website

In [13]:
"""
Input parameters for the 3 part run.
    data_folder - mentions the folder location where data to and from is accessed
    csvfile - name of the merged csv file containing all the parsed data
    url_used - URL of the webite scraped
    brands_regex - RegEx to search relevant chocolate brands
    
@Author: Dhawal Priyadarshi
@Created On: Jan 2019
"""

#*********** User Input Parameters  ***********
# Path to data folder
# data_folder = os.getcwd() + "\\data\\"     # for Windows
data_folder = os.getcwd() + '/data/'         # for Mac and Linux
# File for data storage
csvfile="chocolates_master_file.csv"
# URL of the USDA website (chocolate lookup)
url_used ="https://ndb.nal.usda.gov/ndb/search/list?fgcd=&manu=&lfacet=&count=&max=25&sort=default&\
qlookup=chocolate&offset=0&format=Full&new=&measureby=&ds=&order=asc&qt=&qp=&qa=&qn=&q=&ing="
# Brands search RegEx
brands_regex = r"ghirardelli*|lindt*|mondelez*|mars*|hersheys*|taza*"
#**********************************************

print("Parameters are set!")

Parameters are set!


In [14]:
"""
Extracts data-rows from USDA website on even and odd positions for each page out of the 511 pages and appends 
the data to a list, which is then written into the csv file row by row

@Author: Mansi Nagraj
@Created On: Jan 2019
"""

# Setting run variables
filepath = data_folder + csvfile
offset=0
myWebData = []
http = urllib3.PoolManager()
urllib3.disable_warnings()     # disable SSL warnings

try:
    print("Writing file to", filepath)
    with open(filepath, "a") as output:
        writer = csv.writer(output, dialect='excel')
        # Writing header for the csv file:    
        writer.writerow(["Db","Ndb Id","FoodDescription","Manufacturer"])
        for i in range(1,511):
            new_url= url_used.replace(url_used[113:114] ,str(offset))
            offset=offset+25
            req = http.request('GET', new_url)
            soup = BeautifulSoup(req.data, 'html.parser')
            even_titles = soup.find_all('tr', {"class":"even"})
            odd_titles = soup.find_all('tr', {"class":"odd"})
            # Collecting data for even-numbered rows:        
            for even_title in even_titles:
                cols= even_title.find_all('td')
                myWebData = []
                for col in cols:
                    row_data =col.get_text()
                    evenRowdData= row_data.strip()
                    myWebData.append(evenRowdData)
                writer.writerow(myWebData)
            # Collecting data for odd-numbered rows:             
            for odd_title in odd_titles:
                cols= odd_title.find_all('td')
                myWebData = []
                for col in cols:
                    row_data =col.get_text()
                    oddRowData= row_data.strip()
                    myWebData.append(oddRowData)
                writer.writerow(myWebData)
        print("Scraping completed!")
except BaseException as e:
    print("Base Exception: %s" % str(e))

Writing file to /Users/dhawal/proj/Scraping/data/chocolates_master_file.csv
Scraping completed!


In [15]:
# Displaying the scraped dataset
sitedf = pd.read_csv(filepath)
sitedf

,Db,Ndb Id,FoodDescription,Manufacturer
0,BF,45157421,"CHOCOLATE CHOCOLATE CHOCOLATE, DARK CHOCOLATE ...",Chocolate Chocolate Chocolate
1,BF,45148721,"CHOCOLATE CHOCOLATE CHOCOLATE, WHITE CHOCOLATE...",Chocolate Chocolate Chocolate
2,BF,45286717,"MILK CHOCOLATE, DARK CHOCOLATE, CARAMEL & MILK...",none
3,BF,45148724,"CHOCOLATE CHOCOLATE CHOCOLATE, ALMOND TOFFEE B...",Chocolate Chocolate Chocolate
4,BF,45136309,"CHOCOLATE COOKIES WITH CHOCOLATE CHIPS, UPC: 6...",Dad's Cookie Co.
5,BF,45149559,"GHIRARDELLI CHOCOLATE, CHOCOLATE SQUARES, UPC:...",GHIRARDELLI CHOCOLATE COMPANY
6,BF,45007105,"CHOCOLATE CORTES, SWEET CHOCOLATE, UPC: 073693...",Sucesores Pedro Cortes Inc.
7,BF,45013346,"SHOPRITE, CHOCOLATE SYRUP, CHOCOLATE, UPC: 041...",WAKEFERN FOOD CORPORATION
8,BF,45151749,"ELMER CHOCOLATE, ASSORTED CHOCOLATES, UPC: 041...",ELMER CANDY CORPORATION
9,BF,45143245,"ELMER CHOCOLATE, ASSORTED CHOCOLATES, UPC: 041...",ELMER CANDY CORPORATION


In [16]:
"""
Subsetting the data using user-defined RegEx

@Author: Dhawal Priyadarshi, Mansi Nagraj
@Created On: Jan 2019
"""

all_match = list(map(re.search, repeat(brands_regex\
                                       , len(sitedf['Manufacturer'])), sitedf['Manufacturer'] \
                                       , repeat(re.IGNORECASE, len(sitedf['Manufacturer']))))

s = np.bitwise_not(pd.isnull(all_match))
sitedf_filtered = sitedf[s]
sitedf_filtered.head(15)

,Db,Ndb Id,FoodDescription,Manufacturer
5,BF,45149559,"GHIRARDELLI CHOCOLATE, CHOCOLATE SQUARES, UPC:...",GHIRARDELLI CHOCOLATE COMPANY
14,BF,45364413,"MILK CHOCOLATE, DARK CHOCOLATE, MILK CHOCOLATE...",Mars Chocolate North America LLC
28,BF,45206369,"TAZA CHOCOLATE, CHOCOLATE BAR, UPC: 898456001081",TAZA CHOCOLATE
31,BF,45318593,"DARK CHOCOLATE CHOCOLATE CANDES, UPC: 04045805",Mars Chocolate North America LLC
33,BF,45161032,"GHIRARDELLI CHOCOLATE, CHOCOLATE, ASSORTED, UP...",GHIRARDELLI CHOCOLATE COMPANY
43,BF,45318613,"CHOCOLATE CANDIES DARK CHOCOLATE, UPC: 0400004...",Mars Chocolate North America LLC
45,BF,45318362,"CHOCOLATE CANDIES, DARK CHOCOLATE, UPC: 040000...",Mars Chocolate North America LLC
48,BF,45166054,"GHIRARDELLI CHOCOLATE, MILK CHOCOLATE, UPC: 74...",GHIRARDELLI CHOCOLATE COMPANY
54,BF,45149557,"GHIRARDELLI CHOCOLATE, SPRING IMPRESSIONS CHOC...",GHIRARDELLI CHOCOLATE COMPANY
58,BF,45153637,"GHIRARDELLI CHOCOLATE, WINTER WHITE HOLIDAY SQ...",GHIRARDELLI CHOCOLATE COMPANY


In [17]:
# Randomly selecting 20 rows from the filtered data-frame
# final_sample = sitedf_filtered.sample(20)

# Using fixed Ndb IDs for the purpose of demonstration (based on the downloaded csv files in "data" folder)
filterList = ['45318621','45145291','45318334','45053347','45148117','45376125','45375537','45318454','45369935'\
         ,'45375781','45375900','45158103','45153224','45236273','45375862','45331173','45143068','45004850'\
         ,'45158934','45208905']
final_sample = sitedf_filtered[sitedf_filtered['Ndb Id'].isin(filterList)]
final_sample

,Db,Ndb Id,FoodDescription,Manufacturer
1012,BF,45318621,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC
1095,BF,45318454,"CHOCOLATE CANDIES, UPC: 040000483632",Mars Chocolate North America LLC
1576,BF,45236273,"WEIS QUALITY, 1% LOW FAT CHOCOLATE MILK, CHOCO...","WEIS MARKETS, INC."
2305,BF,45331173,"MILK CHOCOLATE TRUFFLES, UPC: 009542034610",Lindt & Sprungli (Schweiz) AG
3027,BF,45158103,"SOUTHERN HOME, CHOCOLATE STARS, UPC: 607880038897","Bruno's Supermarkets, Inc."
4061,BF,45158934,"GHIRARDELLI CHOCOLATE, MINIS ASSORTMENT, UPC: ...",GHIRARDELLI CHOCOLATE COMPANY
4502,BF,45148117,"KINGS, GRAHAMS, DARK CHOCOLATE, UPC: 073866104779",Kings Super Markets Inc
5460,BF,45004850,"WEIS, SANDWICH KREMES, CHOCOLATE, UPC: 0414975...","WEIS MARKETS, INC."
5643,BF,45053347,"DARK CHOCOLATE NON PAREILS, UPC: 041497499458","WEIS MARKETS, INC."
5690,BF,45208905,"PREMIUM MILK CHOCOLATE BAR, UPC: 086854060049",Ingles Markets Inc.


## Part II - Reading data from CSV files

In [18]:
def NutrientFunction(fn):
    """
    Function to extract the Nutrient data from the different csv files and return parsed dataset
    
    @Author: Mansi Nagraj
    @Created On: Jan 2019
    """
    rows=[]
    inp=[]
    filename = data_folder + str(fn)+'.csv'
    with open(filename , newline='') as csvfile:
        inputfile = csv.reader(csvfile)
        for row in inputfile:
            inp.append(row)
        mylist=[8,11,16,17,19]
        nutrientlist = []
        for n in mylist:
            nut = inp[n][:6]
            nutrientlist.append(nut)
        nutrientdf=pd.DataFrame(nutrientlist,columns=["Nutrient","Unit","DataPoint","StdError","Weight","Value"])
    nutrientdf['Ndb Id']= fn
    return nutrientdf

# Gathering the parsed data from each CSV file into one dataframe
all_results = pd.DataFrame(columns=["Nutrient","Unit","Value","Ndb Id"])
for n in final_sample['Ndb Id']:
    results = NutrientFunction(n)
    all_results = all_results.append(results)
    
all_results.head(15)

/Users/dhawal/proj/Scraping/venv/lib/python3.7/site-packages/pandas/core/frame.py:6701: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,DataPoint,Ndb Id,Nutrient,StdError,Unit,Value,Weight
0,--,45318621,Energy,--,kcal,476,200
1,--,45318621,"Carbohydrate, by difference",--,g,73.81,31.00
2,--,45318621,"Iron, Fe",--,mg,0.86,0.36
3,--,45318621,"Sodium, Na",--,mg,71,30
4,--,45318621,"Vitamin A, IU",--,IU,238,100
0,--,45318454,Energy,--,kcal,524,220
1,--,45318454,"Carbohydrate, by difference",--,g,57.14,24.00
2,--,45318454,"Iron, Fe",--,mg,0.86,0.36
3,--,45318454,"Sodium, Na",--,mg,214,90
4,--,45318454,"Vitamin A, IU",--,IU,238,100


In [27]:
# Merging scraped and file based data into one dataframe

# Branded Food DB
bfdb = pd.concat([final_sample.set_index('Ndb Id'),all_results.set_index('Ndb Id')], axis=1, join='inner')
bfdb

,Db,FoodDescription,Manufacturer,DataPoint,Nutrient,StdError,Unit,Value,Weight
Ndb Id,,,,,,,,,
45318621,BF,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC,--,Energy,--,kcal,476,200
45318621,BF,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC,--,"Carbohydrate, by difference",--,g,73.81,31.00
45318621,BF,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC,--,"Iron, Fe",--,mg,0.86,0.36
45318621,BF,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC,--,"Sodium, Na",--,mg,71,30
45318621,BF,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC,--,"Vitamin A, IU",--,IU,238,100
45318454,BF,"CHOCOLATE CANDIES, UPC: 040000483632",Mars Chocolate North America LLC,--,Energy,--,kcal,524,220
45318454,BF,"CHOCOLATE CANDIES, UPC: 040000483632",Mars Chocolate North America LLC,--,"Carbohydrate, by difference",--,g,57.14,24.00
45318454,BF,"CHOCOLATE CANDIES, UPC: 040000483632",Mars Chocolate North America LLC,--,"Iron, Fe",--,mg,0.86,0.36
45318454,BF,"CHOCOLATE CANDIES, UPC: 040000483632",Mars Chocolate North America LLC,--,"Sodium, Na",--,mg,214,90


# Part III - Munging using Twitter API

In [29]:
"""
Query creator for Twitter API. Creates a query column in the bfdb dataset.
    
@Author: Dhawal Priyadarshi
@Created On: Jan 2019
"""

bfdb['Query'] = bfdb.apply(lambda row: row['FoodDescription'].split(",")[0], axis=1)
bfdb

,Db,FoodDescription,Manufacturer,DataPoint,Nutrient,StdError,Unit,Value,Weight,Query
Ndb Id,,,,,,,,,,
45318621,BF,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC,--,Energy,--,kcal,476,200,CHOCOLATE CANDIES
45318621,BF,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC,--,"Carbohydrate, by difference",--,g,73.81,31.00,CHOCOLATE CANDIES
45318621,BF,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC,--,"Iron, Fe",--,mg,0.86,0.36,CHOCOLATE CANDIES
45318621,BF,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC,--,"Sodium, Na",--,mg,71,30,CHOCOLATE CANDIES
45318621,BF,"CHOCOLATE CANDIES, UPC: 040000249085",Mars Chocolate North America LLC,--,"Vitamin A, IU",--,IU,238,100,CHOCOLATE CANDIES
45318454,BF,"CHOCOLATE CANDIES, UPC: 040000483632",Mars Chocolate North America LLC,--,Energy,--,kcal,524,220,CHOCOLATE CANDIES
45318454,BF,"CHOCOLATE CANDIES, UPC: 040000483632",Mars Chocolate North America LLC,--,"Carbohydrate, by difference",--,g,57.14,24.00,CHOCOLATE CANDIES
45318454,BF,"CHOCOLATE CANDIES, UPC: 040000483632",Mars Chocolate North America LLC,--,"Iron, Fe",--,mg,0.86,0.36,CHOCOLATE CANDIES
45318454,BF,"CHOCOLATE CANDIES, UPC: 040000483632",Mars Chocolate North America LLC,--,"Sodium, Na",--,mg,214,90,CHOCOLATE CANDIES


In [90]:
"""
Twitter querying classes.
    
@Author: Dhawal Priyadarshi
@Created On: Jan 2019
@Credits: VP Russo: https://github.com/vprusso/youtube_tutorials/tree/master/twitter_python
"""
# # # # TWITTER AUTHENTICATOR CLASS# # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        """
        Twitter API authenticator function
        
        @Author: VP Russo: https://github.com/vprusso/youtube_tutorials/tree/master/twitter_python
        """
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

# # # # TWEET QUERYING CLASS # # # #
class TweetQuery():
    """
    This class runs search queries using Tweepy API
    """
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()

    """
    Takes filename, query string, and optional count of number of tweets.
    Appends the tweets to a given file name
    
    @Author: Dhawal Priyadarshi
    @Credits: VPRusso: https://github.com/vprusso/youtube_tutorials/tree/master/twitter_python
    @Credits: VickyQian: https://gist.github.com/vickyqian/f70e9ab3910c7c290d9d715491cde44c
    """
    def search(self, ndbid, query, count = 15):
        auth = self.twitter_authenticator.authenticate_twitter_app()
        api = API(auth)
        tweet_df = pd.DataFrame(columns=['Ndb Id','Tweet_Created','Tweet_Text'])
        try:
            for tweet in Cursor(api.search, q = query, count = count, tweet_mode = 'extended', lang = 'en').items():
                temp_dict = {
                    'Ndb Id': ndbid,
                    'Tweet_Created': tweet.created_at,
                    'Tweet_Text': tweet.full_text
                }
                tweet_df = tweet_df.append(temp_dict, ignore_index=True)
            return tweet_df

        except TweepError as e:
            print(e.response.text, "Going to sleep for 15 mins")
            time.sleep(60 * 15)
        return None
    
print("Classes defined!")

Classes defined!


In [91]:
# Creating data frame for queries

queries = pd.DataFrame(bfdb['Query'])
queries = queries.drop_duplicates()
queries

,Query
Ndb Id,
45318621,CHOCOLATE CANDIES
45236273,WEIS QUALITY
45331173,MILK CHOCOLATE TRUFFLES
45158103,SOUTHERN HOME
45158934,GHIRARDELLI CHOCOLATE
45148117,KINGS
45004850,WEIS
45053347,DARK CHOCOLATE NON PAREILS
45208905,PREMIUM MILK CHOCOLATE BAR


In [92]:
# Hitting the Twitter API with the query strings in the Query dataframe

tweets_all = pd.DataFrame(columns=['Ndb Id','Tweet_Created','Tweet_Text'])
max_query_count = 100       # putting a hard limit to number of requests to avoid API rate_limit error
t_query = TweetQuery()
for idx, val in enumerate(queries.index.values):
    print("Iteration:", idx, val)
    if i <= max_query_count:
        query = queries.loc[val,'Query']     # val is the index/NDB ID
        ndbid = val
        tweets = t_query.search(ndbid, query, count = 10)
        tweets_all = tweets_all.append(tweets, ignore_index=True)
tweets_all

Iteration: 0 45318621
Iteration: 1 45236273
Iteration: 2 45331173
Iteration: 3 45158103
Iteration: 4 45158934
{"errors":[{"message":"Rate limit exceeded","code":88}]} Going to sleep for 15 mins


KeyboardInterrupt: 

In [96]:
# Storing as CSV and displaying tweets pasrsed table

tweets_all.to_csv(data_folder + 'tweets_parsed.csv')
tweets_all

,Ndb Id,Tweet_Created,Tweet_Text
0,45318621,2019-01-31 00:11:43,Can a bunch of chocolate candies with brandy i...
1,45318621,2019-01-31 00:05:12,"For those of you that don’t know, they are cho..."
2,45318621,2019-01-30 23:54:46,That concludes this month’s snacking adventure...
3,45318621,2019-01-30 22:57:29,@mewingwang @NWF8334 @TiffaniAvatar give him a...
4,45318621,2019-01-30 21:40:52,Have a friend coming over for dessert. I want ...
5,45318621,2019-01-30 21:28:25,RT @AddysonSchaffer: Like if you ever go to a ...
6,45318621,2019-01-30 21:10:26,Like if you ever go to a gas station &amp; you...
7,45318621,2019-01-30 21:10:19,small chocolate candies are helpful when learn...
8,45318621,2019-01-30 20:57:44,RT @JenniferOsmond3: #çarpışma They are #çarp...
9,45318621,2019-01-30 20:46:53,RT @FairfaxCSB: 👀Look for Golden Tickets! To c...
